<h4>ワンホットエンコーディング（ダミー変数）</h4>

In [1]:
# ワンホットエンコーディングは、統計で用いられるダミーエンコーディングによく似ているが、まったく同じではない
# 統計では、k個の異なる値をとるカテゴリ特徴量をk-1個の特徴量で表現する（最後の一つはすべてが0として表現される）
# これは、解析を容易にするため（技術的に言うと、これはデータ行列のランク不足を避けるため）だ
import pandas as pd
import os
import mglearn

adult_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
data = pd.read_csv(adult_path, header = None, index_col = False,
                   names = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship",
                            "race","gender","capital-gain","capital-loss","hours-per-week","native-country","income"])
data = data[["age","workclass","education","gender","hours-per-week","occupation","income"]]
display(data.head())

print(data.gender.value_counts(), "\n")
print(data.workclass.value_counts(),"\n")
print(data.education.value_counts(), "\n")
print(data.occupation.value_counts(), "\n")
print(data.income.value_counts(), "\n")
print("Original features:\n", list(data.columns), "\n")
data_dummies = pd.get_dummies(data)
print("Features after get_dummies:\n", list(data_dummies.columns))
display(data_dummies.head())

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


 Male      21790
 Female    10771
Name: gender, dtype: int64 

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64 

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64 

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners 

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,50,13,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,38,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,53,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,28,40,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [2]:
# 訓練セットとテストセットの特徴量の数が異なる場合、訓練セットで学習したモデルをテストセットに適用できないことになってしまう
# 訓練データとテストデータの双方が入ったDataFrameに対してget_dummiesを呼ぶようにするか、訓練データとテストデータに対して
# get_dummiesを呼んだ後に、同じカラム名を含んでいるかを確認して、両方が同じ意味になるようにすればよい
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

features = data_dummies.loc[:, "age":"occupation_ Transport-moving"]
X = features.values
y = data_dummies["income_ >50K"].values
print(f"X.shape: {X.shape} y.shape: {y.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logreg = LogisticRegression(max_iter = 10000).fit(X_train, y_train)
print(f"Test score: {logreg.score(X_test,y_test):.2f}")

X.shape: (32561, 44) y.shape: (32561,)
Test score: 0.81


<h4>数値でエンコードされているカテゴリ</h4>

In [3]:
# カテゴリ特徴量が整数でエンコードされていることは多い。数値で表されているからといって、連続値として扱ってはいけない
# 整数特徴量が連続値を表しているのか離散値やワンホットエンコーディングを表しているのか、明確でない場合も多い
# 例えば星5つによる評価などの場合には、取るべきエンコーディング方法は、個々のタスクや利用する機械学習アルゴリズムに依存する
# pandasのget_dummies関数では、すべての数値を連続値として扱い、ダミー変数を作らない
# これを回避するためには、連続値と離散値を指定することができるscikit-learnのOneHotEncoderを用いるか、DataFrameの列を数値から文字列に変換すればよい
demo_df = pd.DataFrame({"Integer Feature":[0,1,2,1], "Categorical Feature":["socks","fox","socks","box"]})
display(demo_df)
display(pd.get_dummies(demo_df))
demo_df["Integer Feature"] = demo_df["Integer Feature"].astype(str)
display(pd.get_dummies(demo_df, columns = ["Integer Feature", "Categorical Feature"]))

,Integer Feature,Categorical Feature
0,0,socks
1,1,fox
2,2,socks
3,1,box


,Integer Feature,Categorical Feature_box,Categorical Feature_fox,Categorical Feature_socks
0,0,0,0,1
1,1,0,1,0
2,2,0,0,1
3,1,1,0,0


,Integer Feature_0,Integer Feature_1,Integer Feature_2,Categorical Feature_box,Categorical Feature_fox,Categorical Feature_socks
0,1,0,0,0,0,1
1,0,1,0,0,1,0
2,0,0,1,0,0,1
3,0,1,0,1,0,0
